In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import os
from imblearn.over_sampling import SMOTE

In [3]:
# Load in your data here
data = pd.read_csv("/home/novo/icr-2025-psych/data/Combination-Data-Files/AEImoodhc_allPSD.csv") #file name here
print(data.shape[1])
print(data.shape[1])
print(data.iloc[:, 117])

118
118
0      1
1      1
2      1
3      1
4      1
      ..
350    0
351    0
352    0
353    0
354    0
Name: Class, Length: 355, dtype: int64


In [4]:
bands = [
    ('Delta', data.iloc[:, list(range(0,22))+[-1]]),
    ('Theta', data.iloc[:, list(range(0,3))+list(range(22,41))+[-1]]),
    ('Alpha', data.iloc[:, list(range(0,3)) + list(range(41,60))+[-1]]),
    ('Beta', data.iloc[:, list(range(0,3)) + list(range(60,79))+[-1]]),
    ('HighBeta', data.iloc[:, list(range(0,3)) + list(range(79,98))+[-1]]),
    ('Gamma', data.iloc[:, list(range(0,3)) + list(range(98,117))+[-1]]),
    ('All', data.iloc[:, list(range(0,118))])
]
all_model_data = []

#bands = [delta, theta, alpha, beta, highbeta, gamma, allb]

for label, band in bands:
    # Data preprocessing
    print(f"\n{label} columns:\n", band.columns.tolist())
    X = band.drop('Class', axis=1)
    y = band['Class']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, stratify=None)
    #using smote to oversample
    #smote = SMOTE(random_state=0)
    #X_train, y_train = smote.fit_resample(X, y)
    #y_train.hist()
    pca = PCA(0.95)
    X_pca = pca.fit_transform(X)
    X_pca.shape
    X = X.dropna()
    X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.1, random_state=30)
    #using smote to oversample
    #smote = SMOTE(random_state=0)
    #X_train_pca, y_train = smote.fit_resample(X_pca, y)
    #y_train.hist()
    svclassifier = SVC(kernel='linear',probability=True)
    svclassifier.fit(X_train, y_train)
    y_pred = svclassifier.predict(X_test)
    print('SVM accuracy:', svclassifier.score(X_test, y_test))
    print('SVM classification report:\n', classification_report(y_test, y_pred))
    y_scores = svclassifier.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    # Calculate AUC
    svm_auc = auc(fpr, tpr)
    print('SVM AUC value:', svm_auc)
    newsvm=SVC(kernel='linear', probability=True)
    newsvm.fit(X_train_pca, y_train)
    y_pred = newsvm.predict(X_test_pca)
    print('\nSVM accuracy for PCA:', newsvm.score(X_test_pca, y_test))
    print('SVM classification report for PCA:\n', classification_report(y_test, y_pred))
    y_scores = newsvm.predict_proba(X_test_pca)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    pcasvm_auc = auc(fpr, tpr)
    print('SVM AUC value for PCA:', pcasvm_auc)
    
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(X_train, y_train)
    print('KNN accuracy:', knn.score(X_test, y_test))
    y_pred = knn.predict(X_test)
    print('KNN classification report:\n',classification_report(y_test, y_pred))
    y_scores = knn.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    knn_auc = auc(fpr, tpr)
    print('KNN AUC value:', knn_auc)
    pknn = KNeighborsClassifier(n_neighbors=10)
    pknn.fit(X_train_pca, y_train)
    print('\nKNN accuracy for PCA:', pknn.score(X_test_pca, y_test))
    y_pred = pknn.predict(X_test_pca)
    print('KNN classification report for PCA\n', classification_report(y_test, y_pred))
    y_scores = pknn.predict_proba(X_test_pca)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    pcaknn_auc = auc(fpr, tpr)
    print('KNN AUC value for PCA',pcaknn_auc)
    
    model = RandomForestClassifier(n_estimators=40)
    model.fit(X_train, y_train)
    print('Random Forest accuracy:', model.score(X_test, y_test))
    y_pred = model.predict(X_test)
    print('Random Forest classification report\n',classification_report(y_test, y_pred))
    y_scores = model.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    rf_auc = auc(fpr, tpr)
    print('Random Forest AUC value:', rf_auc)
    pmodel = RandomForestClassifier(n_estimators=40)
    pmodel.fit(X_train_pca, y_train)
    print('\nRandom Forest accuracy for PCA:', pmodel.score(X_test_pca, y_test))
    y_pred = pmodel.predict(X_test_pca)
    print('Random Forest classification report for PCA:\n',classification_report(y_test, y_pred))
    y_scores = pmodel.predict_proba(X_test_pca)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    pcarf_auc = auc(fpr, tpr)
    print('Random Forest AUC value for PCA:', pcarf_auc)
    
    lg = LogisticRegression()
    lg.fit(X_train, y_train)
    print('Logistic Regression accuracy:', lg.score(X_test, y_test))
    y_pred = lg.predict(X_test)
    print('Logistic Regression classification report:\n',classification_report(y_test, y_pred))
    y_scores = lg.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    lg_auc = auc(fpr, tpr)
    print('Logistic Regression AUC value:',lg_auc)
    plg = LogisticRegression()
    plg.fit(X_train_pca, y_train)
    print('\nLogistic Regression accuracy for PCA:',plg.score(X_test_pca, y_test))
    y_pred = plg.predict(X_test_pca)
    print('Logistic Regression classification report for PCA:\n',classification_report(y_test, y_pred))
    y_scores = plg.predict_proba(X_test_pca)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    pcalg_auc = auc(fpr, tpr)
    print('Logistic Regression AUC value for PCA',pcalg_auc)
    
    clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
    clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    y_pred = clf.predict(X_test)
    print('XGB accuracy:',clf.score(X_test, y_test))
    print('XGB Classification report:\n',classification_report(y_test, y_pred))
    y_scores = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    xgb_auc = auc(fpr, tpr)
    print('XGB AUC value:',xgb_auc)
    pxgb = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
    pxgb.fit(X_train_pca, y_train, eval_set=[(X_test_pca, y_test)])
    print('\nXGB accuracy for PCA:', pxgb.score(X_test_pca, y_test))
    y_pred = pxgb.predict(X_test_pca)
    print('XGB classification report for PCA:\n',classification_report(y_test, y_pred))
    y_scores = pxgb.predict_proba(X_test_pca)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    pxgb_auc = auc(fpr, tpr)
    print('XBG AUC for PCA:', pxgb_auc)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    model_row = [
        f"Mood_AEI_{label}_PSD",
        svclassifier.score(X_test, y_test), svm_auc,
        knn.score(X_test, y_test), knn_auc,
        model.score(X_test, y_test), rf_auc,
        lg.score(X_test, y_test), lg_auc,
        clf.score(X_test, y_test), xgb_auc
    ]
    pca_row = [
        f"Mood_AEI_{label}_PSDPCA", 
        newsvm.score(X_test_pca, y_test), pcasvm_auc,
        pknn.score(X_test_pca, y_test),pcaknn_auc,
        pmodel.score(X_test_pca, y_test),pcarf_auc,
        plg.score(X_test_pca, y_test),pcalg_auc,
        pxgb.score(X_test_pca, y_test), pxgb_auc
    ]
    all_model_data.append(model_row)
    all_model_data.append(pca_row)
    for row in all_model_data:
        print(row, type(row), len(row) if hasattr(row, '__len__') else 'Not iterable')
columns = ['Combination', 'SVM Accuracy', 'SVM AUC', 
           'KNN Accuracy', 'KNN AUC', 
           'Random Forest Accuracy','Random Forest AUC', 
           'Logistic Regression Accuracy', 'Logistic Regression AUC', 
           'XGB Accuracy', 'XGB AUC']
    
df = pd.DataFrame(all_model_data, columns=columns)
df.to_csv('extradata.csv', mode='a',header=not os.path.exists('extradata.csv'),index=False)


Delta columns:
 ['age', 'education', 'IQ', 'AB.A.delta.a.FP1', 'AB.A.delta.b.FP2', 'AB.A.delta.c.F7', 'AB.A.delta.d.F3', 'AB.A.delta.e.Fz', 'AB.A.delta.f.F4', 'AB.A.delta.g.F8', 'AB.A.delta.h.T3', 'AB.A.delta.i.C3', 'AB.A.delta.j.Cz', 'AB.A.delta.k.C4', 'AB.A.delta.l.T4', 'AB.A.delta.m.T5', 'AB.A.delta.n.P3', 'AB.A.delta.o.Pz', 'AB.A.delta.p.P4', 'AB.A.delta.q.T6', 'AB.A.delta.r.O1', 'AB.A.delta.s.O2', 'Class']
SVM accuracy: 0.8333333333333334
SVM classification report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.83      1.00      0.91        30

    accuracy                           0.83        36
   macro avg       0.42      0.50      0.45        36
weighted avg       0.69      0.83      0.76        36

SVM AUC value: 0.3388888888888889


/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)


SVM accuracy for PCA: 0.9166666666666666
SVM classification report for PCA:
               precision    recall  f1-score   support

           0       0.67      1.00      0.80         6
           1       1.00      0.90      0.95        30

    accuracy                           0.92        36
   macro avg       0.83      0.95      0.87        36
weighted avg       0.94      0.92      0.92        36

SVM AUC value for PCA: 0.9944444444444445
KNN accuracy: 0.6944444444444444
KNN classification report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.81      0.83      0.82        30

    accuracy                           0.69        36
   macro avg       0.40      0.42      0.41        36
weighted avg       0.67      0.69      0.68        36

KNN AUC value: 0.4916666666666666

KNN accuracy for PCA: 0.7777777777777778
KNN classification report for PCA
               precision    recall  f1-score   support


/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precis


XGB accuracy for PCA: 0.9166666666666666
XGB classification report for PCA:
               precision    recall  f1-score   support

           0       0.80      0.67      0.73         6
           1       0.94      0.97      0.95        30

    accuracy                           0.92        36
   macro avg       0.87      0.82      0.84        36
weighted avg       0.91      0.92      0.91        36

XBG AUC for PCA: 0.9388888888888889
[[ 4  2]
 [ 1 29]]
['Mood_AEI_Delta_PSD', 0.8333333333333334, 0.3388888888888889, 0.6944444444444444, 0.4916666666666666, 0.6944444444444444, 0.48888888888888893, 0.8333333333333334, 0.7111111111111111, 0.8333333333333334, 0.6638888888888889] <class 'list'> 11
['Mood_AEI_Delta_PSDPCA', 0.9166666666666666, 0.9944444444444445, 0.7777777777777778, 0.8527777777777779, 0.8611111111111112, 0.9083333333333333, 0.9444444444444444, 1.0, 0.9166666666666666, 0.9388888888888889] <class 'list'> 11

Theta columns:
 ['age', 'education', 'IQ', 'AB.B.theta.a.FP1', 'AB.B

...

KNN AUC value: 0.5611111111111111

KNN accuracy for PCA: 0.8611111111111112
KNN classification report for PCA
               precision    recall  f1-score   support

           0       0.56      0.83      0.67         6
           1       0.96      0.87      0.91        30

    accuracy                           0.86        36
   macro avg       0.76      0.85      0.79        36
weighted avg       0.90      0.86      0.87        36

KNN AUC value for PCA 0.95
Random Forest accuracy: 0.8055555555555556
Random Forest classification report
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.83      0.97      0.89        30

    accuracy                           0.81        36
   macro avg       0.41      0.48      0.45        36
weighted avg       0.69      0.81      0.74        36

Random Forest AUC value: 0.6638888888888889

Random Forest accuracy for PCA: 0.9166666666666666
Random Forest classification rep

/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precis


XGB accuracy for PCA: 0.9444444444444444
XGB classification report for PCA:
               precision    recall  f1-score   support

           0       1.00      0.67      0.80         6
           1       0.94      1.00      0.97        30

    accuracy                           0.94        36
   macro avg       0.97      0.83      0.88        36
weighted avg       0.95      0.94      0.94        36

XBG AUC for PCA: 0.9500000000000001
[[ 4  2]
 [ 0 30]]
['Mood_AEI_Delta_PSD', 0.8333333333333334, 0.3388888888888889, 0.6944444444444444, 0.4916666666666666, 0.6944444444444444, 0.48888888888888893, 0.8333333333333334, 0.7111111111111111, 0.8333333333333334, 0.6638888888888889] <class 'list'> 11
['Mood_AEI_Delta_PSDPCA', 0.9166666666666666, 0.9944444444444445, 0.7777777777777778, 0.8527777777777779, 0.8611111111111112, 0.9083333333333333, 0.9444444444444444, 1.0, 0.9166666666666666, 0.9388888888888889] <class 'list'> 11
['Mood_AEI_Theta_PSD', 0.8333333333333334, 0.5166666666666666, 0.8888

/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/novo/miniforge3/envs/smote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precis

[5]	validation_0-logloss:0.32060
[6]	validation_0-logloss:0.31210
[7]	validation_0-logloss:0.32750

XGB accuracy for PCA: 0.8333333333333334
XGB classification report for PCA:
               precision    recall  f1-score   support

           0       0.50      0.33      0.40         6
           1       0.88      0.93      0.90        30

    accuracy                           0.83        36
   macro avg       0.69      0.63      0.65        36
weighted avg       0.81      0.83      0.82        36

XBG AUC for PCA: 0.9444444444444444
[[ 2  4]
 [ 2 28]]
['Mood_AEI_Delta_PSD', 0.8333333333333334, 0.3388888888888889, 0.6944444444444444, 0.4916666666666666, 0.6944444444444444, 0.48888888888888893, 0.8333333333333334, 0.7111111111111111, 0.8333333333333334, 0.6638888888888889] <class 'list'> 11
['Mood_AEI_Delta_PSDPCA', 0.9166666666666666, 0.9944444444444445, 0.7777777777777778, 0.8527777777777779, 0.8611111111111112, 0.9083333333333333, 0.9444444444444444, 1.0, 0.9166666666666666, 0.938888

In [ ]:
psd and fc
bands = [
    ('Delta', data.iloc[:, list(range(0,22))+list(range(118,289))+[-1]]),
    ('Theta', data.iloc[:, list(range(0,3))+list(range(22,41))+list(range(289,460))+[-1]]),
    ('Alpha', data.iloc[:, list(range(0,3)) + list(range(41,60))+list(range(460,631))+[-1]]),
    ('Beta', data.iloc[:, list(range(0,3)) + list(range(60,79))+list(range(631,802))+[-1]]),
    ('HighBeta', data.iloc[:, list(range(0,3)) + list(range(79,98))+list(range(802,973))+[-1]]),
    ('Gamma', data.iloc[:, list(range(0,3)) + list(range(98,117))+list(range(973,1144))+[-1]]),
    ('All', data.iloc[:, list(range(0,117))+list(range(118,1145))])
]